### Ask the GPT API a set of Python Programming Questions 
##### (some from https://www.geeksforgeeks.org/python-programming-examples/)

#### Installation:
Using python 3.9+
1. Create a new Virtualenvs environment using venv (or something similar)
2. From a bash terminal run:
  * source your_virtual_env_folder_path/bin/activate
  * pip install -r your_project_folder/requirements.txt

#### To Run:
1. Make sure you put a valid OpenAI API key in the file ```temp_folder/api_key.txt```
2. Run all cells in this notebook

#### Output:
In the folder ```temp_folder```, there will be 2 files:
1. ```programing_question_responses.json```: This file contains the full json output from GPT for all 129 programming questions
2. ```code_snippits.txt```: This file has just the question text and the response text

In [34]:
import pandas as pd
import numpy as np
import json
import openai
import os
import tqdm
from IPython.display import display
openai.api_key = open('temp_folder/api_key.txt').read()
price_per_1k_tokens = 0.002


### Get all questions from the file ```python_questions.txt```

In [3]:
lines = open('python_questions.txt','r').read().split('\n')
lines = [
    ' '.join(l.strip().split()[1:]).replace('in Python','').replace('Python','').replace('for Program','') for l in lines
    if len(l.strip().split())>5
]
lines = [l.replace(' | ','') for l in lines]
for i in range(len(lines)):
    if 'program' in lines[i].lower():
        if 'python' in lines[i].lower():
            lines[i] = 'Write a ' + lines[i]
        else:
            lines[i] = 'Write a python ' + lines[i]
    elif 'ways' in lines[i].lower():
        lines[i] = 'Using python find ' + lines[i]
    else:
        lines[i] = 'Using python ' + lines[i]
lines


['Write a python  program to check if a string is palindrome or not',
 'Using python Reverse words in a given String ',
 'Using python find Ways to remove i’th character from string ',
 'Using python Check if a Substring is Present in a Given String',
 'Using python find Find length of a string in python (4 ways)',
 'Write a python  program to print even length words in a string',
 'Write a python Program to accept the strings which contains all vowels',
 'Using python Count the Number of matching characters in a pair of string',
 'Write a python  program to count number of vowels using sets in given string',
 'Using python Remove all duplicates from a given string ',
 'Write a python Program to check if a string contains any special character',
 'Using python Generating random strings until a given string is generated',
 'Using python Find words which are greater than given length k',
 'Write a python  program for removing i-th character from a string',
 'Write a python  program to sp

### Simple code to generate code from gpt model

In [17]:
def generate_code(task):
    """
    Generates Python code based on a given task using the OpenAI API.
    """
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "user", "content": task},
        ]
    )    
    return response



#### Run the model on all questions

In [18]:
code_snippits = {}
for line in tqdm.tqdm(lines):
    new_code = generate_code(line)
#     print(new_code)
#     c = input('continue (y/n)?')
#     if c.lower()!='y':
#         break
    code_snippits[line] = new_code

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 129/129 [46:39<00:00, 21.70s/it]


### Calculate the cost of this operation

In [21]:
all_keys = list(code_snippits.keys())

In [29]:
total_cost = sum([
    int(v['usage']['total_tokens'])
    for v in code_snippits.values()
])/1000 * price_per_1k_tokens
print(
    f"Total cost for code completion of {len(code_snippits)} answers to the programming questions = {total_cost}")

Total cost for code completion of 129 answers to the programming questions = 0.073994


### Display a sample response from the gpt model

In [30]:
code_snippits[all_keys[0]]

<OpenAIObject chat.completion id=chatcmpl-7CTBcqHYAm6UteKRlRx2tAHdbjZ86 at 0x11e752130> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Here's a Python program to check if a string is palindrome or not:\n\n```python\ndef is_palindrome(s):\n    # Remove all non-alphanumeric characters and convert the string to lowercase\n    s = ''.join(ch for ch in s if ch.isalnum()).lower()\n    \n    # Check if the reversed string is the same as the original string\n    return s == s[::-1]\n\n# Test the function\nprint(is_palindrome('A man, a plan, a canal: Panama'))  # True\nprint(is_palindrome('race a car'))  # False\n```\n\nIn this program, we define a function `is_palindrome` that takes a string as input and returns `True` if the string is a palindrome and `False` otherwise. To check if a string is a palindrome, we remove all non-alphanumeric characters (using the `isalnum` method) and convert the string to lowercase. Then we ch

### Write all responses to a json file

In [33]:
json.dump(
    code_snippits,
    open('temp_folder/programing_question_responses.json','w')
)

### Write all Questions and answer to a text file

In [77]:
fname = 'temp_folder/code_snippits.txt'
try:
    open(fname, "a").truncate(0)
except:
    pass
with open(fname,'w') as fp2:
    for k in all_keys:
        fp2.writelines(f"\n{'-'* len(k)}\n")
        fp2.writelines(k)
        fp2.writelines(f"\n{'-'* len(k)}\n\n")
        for cs in code_snippits[k]['choices']:
            fp2.writelines(list(cs['message'].values())[1])    
        fp2.writelines(f"\n{'-'* len(k)}\n\n\n")
